In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Introduction
**Overview of the Dataset**

The Dataset contains 250,000 individuals' data who either payed their loan on time or not indicated by the Risk Flag.

# Questions

1. Professions with highest and lowest average income.
2. Professions with highest and lowest average age.
3. What is the income based on the age?
4. What is the experience according to the age?
5. What is the income according to profession?
6. Do homes with a few years have a higher rental potential than others?
7. Is there a relation between marital status and home ownership?
8. What is the income based on the experience?
9. Average income of Car Owners.
10. Average age of married individuals.
11. The highest ten professions with the income.

# 2. Data Cleaning
Exploring the data, seeing if there is something to trim or clean up.

In [ ]:
def printShapeAndData(panda):  
    print(panda.shape)
    display(panda.head(10))
data = pd.read_csv('/kaggle/input/loan-prediction-based-on-customer-behavior/Training Data.csv')

printShapeAndData(data)

In [ ]:
data.describe()

**Train id is not needed so we remove it.Train id is not needed so we remove it.**

In [ ]:
data.drop(columns =["Id"], inplace = True)

**Checking for Null Values**

In [ ]:
pd.isnull(data).sum()

In [ ]:
df = data.copy()
df

**Checking Correlation**

In [ ]:
Cor=data.corr()
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(), cmap='YlGnBu', annot = True)
plt.title("Correlation Map", fontweight = "bold", fontsize=16)
plt.show()

**Changing Married and Single to 1 and 0 respectively and Car_Ownership  Yes = 1 & no =0**

In [ ]:
df["Married/Single"]= pd.Series(np.where(df["Married/Single"].values =='married', 1, 0), df.index)
df["Car_Ownership"]= pd.Series(np.where(df["Car_Ownership"].values =='yes', 1, 0), df.index)

# 3. Explantory Data Analysis

In [ ]:
#6. Do homes with a few years have a higher rental potential than others?
sns.set_style('whitegrid')
sns.countplot(x='CURRENT_HOUSE_YRS' ,data = data , hue = 'House_Ownership')
plt.show()

In [ ]:
#7. Is there a relation between marital status and home ownership?
sns.set_style('whitegrid')
sns.countplot(x='Married/Single' ,data = data , hue = 'House_Ownership')
plt.show()

In [ ]:
df['Profession'].unique()

In [ ]:
def MinAndMax(dataframe, column, column2):
    #Getting Max Value
    high = dataframe[column].idxmax()
    high_data = pd.DataFrame(df.loc[high])
    #Getting Min Value
    low = dataframe[column].idxmin()
    low_data = pd.DataFrame(df.loc[low])
    dataMinMax = pd.concat([high_data, low_data], axis=1) #creating a table for the detail of both columns
    sChigh = dataMinMax[high][column2]
    sClow = dataMinMax[low][column2]
    return print("The lowest and highest in {} are: {} and {} and their {}s are {} and {}".format(column,low,high,column2,sChigh, sClow)), dataMinMax

    

In [ ]:
#1.Professions with highest and lowest average income.
MinAndMax(df, 'Income','Profession')

In [ ]:
#2.Professions with highest and lowest average age.
MinAndMax(df, 'Age','Profession')

In [ ]:
#3.What is the income based on the age?
q=df.groupby('Age', as_index=False)['Income'].mean()
q.Income= round (q.Income,1)

sns.regplot(data = q, x = 'Age', y = 'Income', fit_reg = False); 

plt.xlabel('Age');
plt.ylabel('Income');

In [ ]:
#4.What is the experience according to the age?
w= df.groupby('Age', as_index=False)['Experience'].mean()
w.Experience= round (w.Experience,1)

sns.regplot(data = w, x = 'Age', y = 'Experience', fit_reg = False); 

plt.xlabel('Age');
plt.ylabel('Experience');

In [ ]:
#5. What is the income according to profession?
z=df.groupby('Profession', as_index=False)['Income'].mean()
z.Income= round (z.Income,1)

sns.regplot(data = z, x = 'Profession', y = 'Income', fit_reg = False); 
plt.xticks(rotation= 90)
plt.xlabel('Profession');
plt.ylabel('Income');

In [ ]:
#8. what is the income based on the experience?
sns.barplot(x ='Experience', y ='Income', data = data)
plt.show()

In [ ]:
#11.The highest ten professions with the income.
heighest_three = df.groupby("Profession")["Income"].sum().sort_values(ascending=False).head(10)
HighestIndex=list(heighest_three.index)
HighestValue=list(heighest_three.values)
sns.barplot(x= HighestValue, y=HighestIndex);

**Checking for Outliers**

In [ ]:
import matplotlib.pyplot as plt

for column in data._get_numeric_data():
    plt.figure()
    data.boxplot([column])

In [ ]:
Y = df['Income'].values
X = df['Experience'].values
plt.scatter(X,Y)

In [ ]:
a = sum([(x - np.mean(X)) * (y - np.mean(Y)) for x, y in zip(X,Y)]) / sum([(x - np.mean(X))**2 for x in X])
b = np.mean(Y) - a * np.mean(X)
yHat = lambda x: a * x + b

In [ ]:
plt.scatter(X,Y)
plt.plot([X.min(), X.max()], [yHat(X.min()), yHat(X.max())], color = 'b')
plt.scatter(X.mean(), Y.mean(), marker = 'D', c = 'r')

In [ ]:
TotalSS = sum([(y - np.mean(Y))**2 for y in Y])
ResidualSS = sum([(y - yHat(x))**2 for x, y in zip(X,Y)])
R2 = 1 - ResidualSS/TotalSS
print("Total Sum Squared is %f, Residual Sum Squared is %f, Fit Quality is %f." % (TotalSS, ResidualSS, R2))

In [ ]:
# 9. Average income of Car Owners.
dt=data.copy()
dt["Car_Ownership"]= pd.Series(np.where(dt["Car_Ownership"].values =='yes', 1, 0), dt.index)
dt[['Car_Ownership']]

In [ ]:
dt = dt.loc[~((dt['Car_Ownership'] == 0)) ]
dt

In [ ]:
car=dt["Car_Ownership"].mean()
car



In [ ]:
income=dt["Income"].mean()
income

In [ ]:
sns.set_style("whitegrid")
sns.barplot(x=["Owned Car", "Income"],y=[car,income])

In [ ]:
df_mean = dt[["Income", "Car_Ownership"]].mean()
df_mean


In [ ]:
df_mean.plot(kind='bar')

In [ ]:
#10. Average age of married individuals
dt["Married/Single"]= pd.Series(np.where(dt["Married/Single"].values =='married', 1, 0), dt.index)
dt[["Married/Single"]]

In [ ]:
dt = dt.loc[~((dt['Married/Single'] == 0)) ]
dt

In [ ]:
married=dt["Married/Single"].mean()
married

In [ ]:
age=dt["Age"].mean()
age

In [ ]:

df_mean = dt[["Age", "Married/Single"]].mean()
df_mean

In [ ]:
sns.set_style("whitegrid")
sns.barplot(x=["Age   ", "Married"],y=[age,married])


In [ ]:
df_mean.plot(kind='bar')
plt.xlabel('Average age of married individuals', fontsize = 15) 

# Conclusion

**The fit Quality is good due to the density of dataset and the correlation between the Income and Experience.
As for the outliers, there is none, indicating that the dataset is done professionally**

# 4. Machine Learning Models for predicting Loan acceptance.

**4.1 From the Correlation Map as seen above, the two columns Experience and CURRENT_JOB_YRS are too related, resulting in
both features having the same value in the same row re-occuring too many times in the datasets, therefore PCA is needed.** 

First, we ready our feautures for PCA by scaling them.

In [ ]:
dfML = df.copy() #Copying just incase.
features = ["CURRENT_JOB_YRS","Experience"]

dfPCA = dfML[features]
scaledDF = (dfPCA - dfPCA.mean(axis=0))/dfPCA.std()
scaledDF

Then we apply PCA to the scaled features.

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(scaledDF)
principalDf = pd.DataFrame(data = principalComponents, columns = ['PC. 1', 'PC. 2'])
principalDf.head()

Adding the PCA features into the dataset

In [ ]:
dfML = pd.concat([df,principalDf],axis=1)
dfML.head()

**4.2 Label Encoding the Categorical Values to numerical ones for Training and Testing**

In [ ]:
features = ['House_Ownership','Profession','CITY','STATE'] #Categorical Values
label_encoder = LabelEncoder()

for col in features:
    dfML[col] = label_encoder.fit_transform(dfML[col])

In [ ]:
dfML

**4.3 Data Splitting**

Removing the target for the training set and to assign it to the test set.

In [ ]:
if 'Risk_Flag' in dfML.columns:
    data_targets=dfML.pop('Risk_Flag')
data_targets

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target = train_test_split(dfML,data_targets, test_size=0.2)

**scalling the data**

feature-wise normalization: for each feature in the input data (a column in the input data matrix), we will subtract the mean of the feature and divide by the standard deviation, so that the feature will be centered around 0 and will have a unit standard deviation.

In [ ]:
mean = train_data.mean()
train_data -= mean
std = train_data.std()
train_data /= std

test_data -= mean
test_data /= std



In [ ]:
printShapeAndData(train_data)

In [ ]:
printShapeAndData(train_target)

In [ ]:
printShapeAndData(test_data)

In [ ]:
printShapeAndData(test_target)

**4.4 Different Machine Learning Models**

***4.4.1 Logistic Regression***

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_predict = LogisticRegression()
logistic_predict.fit(train_data, train_target)
print('Accuracy of Logistic regression classifier on training set: {:.5f}'.format(logistic_predict.score(train_data, train_target)))
print('Accuracy of Logistic regression classifier on test set: {:.5f}'.format(logistic_predict.score(test_data, test_target)))

***4.4.2 Decision Tree Classifier***

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier().fit(train_data, train_target)

print('Accuracy of Decision Tree classifier on training set: {:.5f}'.format(clf.score(train_data, train_target)))
print('Accuracy of Decision Tree classifier on test set: {:.5f}'.format(clf.score(test_data, test_target)))

**4.4.3 Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()
RFC.fit(train_data, train_target)

print('Accuracy of Random Forest Classifier on training set: {:.5f}'.format(RFC.score(train_data, train_target)))
print('Accuracy of Random Forest Classifier on test set: {:.5f}'.format(RFC.score(test_data, test_target)))

**4.4.3 K Neighbors Classifier**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(train_data, train_target)

print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(train_data, train_target)))
print('Accuracy of KNN classifier on test set: {:.2f}'
     .format(knn.score(test_data, test_target)))

# 5. Conclusion

**From the results we can see that the Random Forest Classifier is the best for this particular dataset as it yields the best result in terms of accuracy: 90%**

**5.1 fine tuning the best model to achieve best result**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 2 fold cross validation, 
# search across 2 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, scoring='accuracy', n_iter = 2, cv = 2, verbose=2, n_jobs = -1)

# Fit the random search model
rf_random.fit(train_data, train_target)


In [ ]:
# scores
print(" Results from Random Search " )
print("\n The best estimator across ALL searched params:\n", rf_random.best_estimator_)
print("\n The best score across ALL searched params:\n", rf_random.best_score_)
print("\n The best parameters across ALL searched params:\n", rf_random.best_params_)
print('\n Accuracy of Random Forest Classifier best parameters on test set: {:.5f} \n'.format(rf_random.score(test_data, test_target)))

**5.2 confustion matrix for the model**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
y_pred=rf_random.best_estimator_.predict(test_data)
array=confusion_matrix(test_target, y_pred)

In [ ]:
f = sns.heatmap(array, annot=True, fmt='d')